## Model Deployment


In this experiment, deployment is a phase to simulate the model application in the real case. This notebook contains processes to get an output label of a raw video, whether it is predicted as a deepfake or not. The process in this notebook describes how the model could be applied in the real deployed environment as a deepfake detection system.


In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [2]:
def preprocess_image(image):
    preprocessed_image = image = tf.image.resize(image, size=[128, 128])
    preprocessed_image /= 255
    return preprocessed_image

In [3]:
def crop_face(frame):
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml"
    )

    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_cascade.detectMultiScale(
        grayscale_frame, scaleFactor=1.1, minNeighbors=6
    )

    for x, y, w, h in detected_faces:
        cropped_face = frame[y : y + h, x : x + w]
        return cropped_face

In [4]:
def extract_and_preprocess_frames(video_path, num_output_frames=None):
    extracted_frames = []

    # Extract face frames from the video file
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        cropped_face = crop_face(frame)
        if cropped_face is None:
            continue

        extracted_frames.append(cropped_face)

    cap.release()

    if len(extracted_frames) < 1:
        return []

    # Write num_output_frames extracted frames to output_dir
    saved_frames_count = 0
    if num_output_frames:
        frame_interval = (
            len(extracted_frames) // num_output_frames
            if len(extracted_frames) > num_output_frames
            else 1
        )
    else:
        frame_interval = 1

    preprocessed_data = []
    for i in range(0, len(extracted_frames), frame_interval):
        preprocessed_image = preprocess_image(extracted_frames[i])
        preprocessed_data.append(preprocessed_image)

        if num_output_frames and len(preprocessed_data) >= num_output_frames:
            break

    return np.array(preprocessed_data), len(extracted_frames)

In [5]:
def predict(model, filepath, num_frames=None, mc_dropout=False, mc_sample_size=50):
    filename = os.path.basename(filepath)
    print(f"\nProcessing {filename}...")

    if not os.path.isfile(filepath):
        print(f'Error predicting file. "{filepath}" is not a valid file.')
        return

    frame_data, total_cropped_face = extract_and_preprocess_frames(filepath, num_frames)
    if len(frame_data) < 1:
        print("Error predicting file. No faces detected.")
        return

    if mc_dropout:
        frame_mc_prediction_score = np.stack(
            [model(frame_data, training=True) for _ in range(mc_sample_size)]
        )
        frame_prediction_scores = np.mean(frame_mc_prediction_score, axis=0)
    else:
        frame_prediction_scores = model(frame_data)

    THRESHOLD = 0.5
    video_prediction_score = np.mean(frame_prediction_scores)
    video_prediction_label = (
        "Deepfake" if video_prediction_score < THRESHOLD else "Real"
    )
    video_prediction_probability = (
        1 - video_prediction_score
        if video_prediction_score < THRESHOLD
        else video_prediction_score
    )

    print("\nPrediction Result")
    print(f"Filename         : {filename}")
    print(f"Face Frames      : {total_cropped_face}")
    print(f"Evaluated Frames : {num_frames or total_cropped_face}")
    print(f"Prediction       : {video_prediction_label}")
    print(f"Probability      : {round(video_prediction_probability * 100, 3)}%")
    if mc_dropout:
        print(f"Technique        : MC Dropout Forward Pass")
        print(f"MC Sample Size   : {mc_sample_size}")
    else:
        print(f"Technique        : Regular Forward Pass")

In [6]:
model_path = "/kaggle/input/cnn-model-training/dropout_cnn_cdf_model.keras"
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "dropout_cnn_cdf"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 128, 128, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_1 (MaxPooling2D)    │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 64, 64, 32)     │       204,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_2 (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 32, 32, 160)    │       128,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_3 (MaxPooling2D)    │ (None, 16, 16, 160)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4 (Conv2D)                 │ (None, 16, 16, 128)    │       512,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_4 (MaxPooling2D)    │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_5 (Conv2D)                 │ (None, 8, 8, 192)      │       614,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_5 (MaxPooling2D)    │ (None, 4, 4, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_6 (Conv2D)                 │ (None, 4, 4, 128)      │       614,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_6 (MaxPooling2D)    │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 96)             │         9,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 96)             │         9,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,578,213 (25.09 MB)

 Trainable params: 2,192,737 (8.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,385,476 (16.73 MB)

In [7]:
%%time
filepath = "/kaggle/input/deepfake-detection-test-vids/deepfake-2.mp4"
predict(model, filepath)
print()


Processing deepfake-2.mp4...

Prediction Result
Filename         : deepfake-2.mp4
Face Frames      : 333
Evaluated Frames : 333
Prediction       : Deepfake
Probability      : 60.573%
Technique        : Regular Forward Pass

CPU times: user 3min 50s, sys: 28.5 s, total: 4min 19s
Wall time: 1min 24s


In [8]:
%%time
filepath = "/kaggle/input/deepfake-detection-test-vids/deepfake-2.mp4"
predict(model, filepath, mc_dropout=True, mc_sample_size=50)
print()


Processing deepfake-2.mp4...

Prediction Result
Filename         : deepfake-2.mp4
Face Frames      : 333
Evaluated Frames : 333
Prediction       : Deepfake
Probability      : 60.437%
Technique        : MC Dropout Forward Pass
MC Sample Size   : 50

CPU times: user 3min 51s, sys: 28.7 s, total: 4min 20s
Wall time: 1min 31s


In [9]:
%%time
filepath = "/kaggle/input/deepfake-detection-test-vids/real-2.mp4"
predict(model, filepath)
print()


Processing real-2.mp4...

Prediction Result
Filename         : real-2.mp4
Face Frames      : 300
Evaluated Frames : 300
Prediction       : Real
Probability      : 63.344%
Technique        : Regular Forward Pass

CPU times: user 5min 52s, sys: 44 s, total: 6min 36s
Wall time: 2min 9s


In [10]:
%%time
filepath = "/kaggle/input/deepfake-detection-test-vids/real-2.mp4"
predict(model, filepath, mc_dropout=True, mc_sample_size=50)
print()


Processing real-2.mp4...

Prediction Result
Filename         : real-2.mp4
Face Frames      : 300
Evaluated Frames : 300
Prediction       : Real
Probability      : 62.916%
Technique        : MC Dropout Forward Pass
MC Sample Size   : 50

CPU times: user 5min 46s, sys: 15.8 s, total: 6min 1s
Wall time: 1min 46s
